#Preparation for OpenML AutoML benchmark


##Installing of required packages

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


We need theese things:

boto3>=1.9,<2.0

liac-arff>=2.5,<3.0

numpy>=1.15,<2.0

pandas>=0.23,<2.0

psutil>=5.4,<6.0

ruamel.yaml>=0.15,<1.0

openml==0.11.0

scikit-learn>=0.22.2

In [ ]:
!pip install boto3

     |████████████████████████████████| 133kB 5.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 7.2MB 33.5MB/s 
     |████████████████████████████████| 143kB 37.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install liac-arff

  Created wheel for liac-arff: filename=liac_arff-2.5.0-cp36-none-any.whl size=11733 sha256=d716498adfd34f14f53adfc43054b6051acdcb7b338210a88fda988d6799c236
  Stored in directory: /root/.cache/pip/wheels/77/8d/b4/8bfce5beea9a3496cc15b24961876adb7b6e2912ff09164179
Successfully built liac-arff


In [ ]:
!pip install ruamel.yaml

     |████████████████████████████████| 112kB 4.3MB/s 
     |████████████████████████████████| 552kB 7.1MB/s 


In [ ]:
!pip install openml

     |████████████████████████████████| 112kB 4.3MB/s 
     |████████████████████████████████| 133kB 6.6MB/s 
  Created wheel for openml: filename=openml-0.11.0-cp36-none-any.whl size=127467 sha256=25954afbdf58775ebb60b946beafeaf6e54b7e015e2fabd14bdb32938ac79c9e
  Stored in directory: /root/.cache/pip/wheels/fb/29/f1/58115101bafad19069e838060ab6bdd8046abceba508500e03
Successfully built openml
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.3
    Uninstalling urllib3-1.26.3:
      Successfully uninstalled urllib3-1.26.3


Let's check that all packages are installed correctly

In [ ]:
!pip list -v | grep boto3

boto3                         1.17.3          /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep liac-arff

liac-arff                     2.5.0           /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep psutil

psutil                        5.4.8           /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep numpy

numpy                         1.19.5          /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep pandas

pandas                        1.1.5           /usr/local/lib/python3.6/dist-packages pip      
pandas-datareader             0.9.0           /usr/local/lib/python3.6/dist-packages pip      
pandas-gbq                    0.13.3          /usr/local/lib/python3.6/dist-packages pip      
pandas-profiling              1.4.1           /usr/local/lib/python3.6/dist-packages pip      
sklearn-pandas                1.8.0           /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep ruamel

ruamel.yaml                   0.16.12         /usr/local/lib/python3.6/dist-packages pip      
ruamel.yaml.clib              0.2.2           /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep openml

openml                        0.11.0          /usr/local/lib/python3.6/dist-packages pip      


In [ ]:
!pip list -v | grep scikit-learn

scikit-learn                  0.22.2.post1    /usr/local/lib/python3.6/dist-packages pip      


And now benchmark. Make IS_REP_CLONED = False if it is first time execution of this notebook.


In [ ]:
%cd gdrive/MyDrive/AutoML_benchmark/

/content/gdrive/MyDrive/AutoML_benchmark


In [ ]:
IS_REP_CLONED = True

In [ ]:
if IS_REP_CLONED == False:
    ! git clone https://github.com/openml/automlbenchmark.git

In [ ]:
%cd automlbenchmark/

/content/gdrive/MyDrive/AutoML_benchmark/automlbenchmark


In [ ]:
! ls

amlb		      frameworks	  reports	    runbenchmark.py
aws_monitor.ipynb     LICENSE		  requirements.txt  runscores.py
dev-requirements.txt  logs		  resources	    runstable.sh
docs		      pytest.ini	  results	    scripts
examples	      recover_results.py  runall.py	    tests


#Implementation of TabNet with tensorflow (by Google guys)

In [ ]:
# coding=utf-8
# Copyright 2021 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""TabNet model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf


def glu(act, n_units):
  """Generalized linear unit nonlinear activation."""
  return act[:, :n_units] * tf.nn.sigmoid(act[:, n_units:])


class TabNet(object):
  """TabNet model class."""

  def __init__(self,
               columns,
               num_features,
               feature_dim,
               output_dim,
               num_decision_steps,
               relaxation_factor,
               batch_momentum,
               virtual_batch_size,
               num_classes,
               epsilon=0.00001):
    """Initializes a TabNet instance.
    Args:
      columns: The Tensorflow column names for the dataset.
      num_features: The number of input features (i.e the number of columns for
        tabular data assuming each feature is represented with 1 dimension).
      feature_dim: Dimensionality of the hidden representation in feature
        transformation block. Each layer first maps the representation to a
        2*feature_dim-dimensional output and half of it is used to determine the
        nonlinearity of the GLU activation where the other half is used as an
        input to GLU, and eventually feature_dim-dimensional output is
        transferred to the next layer.
      output_dim: Dimensionality of the outputs of each decision step, which is
        later mapped to the final classification or regression output.
      num_decision_steps: Number of sequential decision steps.
      relaxation_factor: Relaxation factor that promotes the reuse of each
        feature at different decision steps. When it is 1, a feature is enforced
        to be used only at one decision step and as it increases, more
        flexibility is provided to use a feature at multiple decision steps.
      batch_momentum: Momentum in ghost batch normalization.
      virtual_batch_size: Virtual batch size in ghost batch normalization. The
        overall batch size should be an integer multiple of virtual_batch_size.
      num_classes: Number of output classes.
      epsilon: A small number for numerical stability of the entropy calcations.
    Returns:
      A TabNet instance.
    """

    self.columns = columns
    self.num_features = num_features
    self.feature_dim = feature_dim
    self.output_dim = output_dim
    self.num_decision_steps = num_decision_steps
    self.relaxation_factor = relaxation_factor
    self.batch_momentum = batch_momentum
    self.virtual_batch_size = virtual_batch_size
    self.num_classes = num_classes
    self.epsilon = epsilon

  def encoder(self, data, reuse, is_training):
    """TabNet encoder model."""

    with tf.variable_scope("Encoder", reuse=reuse):

      # Reads and normalizes input features.
      features = tf.feature_column.input_layer(data, self.columns)
      features = tf.layers.batch_normalization(
          features, training=is_training, momentum=self.batch_momentum)
      batch_size = tf.shape(features)[0]

      # Initializes decision-step dependent variables.
      output_aggregated = tf.zeros([batch_size, self.output_dim])
      masked_features = features
      mask_values = tf.zeros([batch_size, self.num_features])
      aggregated_mask_values = tf.zeros([batch_size, self.num_features])
      complemantary_aggregated_mask_values = tf.ones(
          [batch_size, self.num_features])
      total_entropy = 0

      if is_training:
        v_b = self.virtual_batch_size
      else:
        v_b = 1

      for ni in range(self.num_decision_steps):

        # Feature transformer with two shared and two decision step dependent
        # blocks is used below.

        reuse_flag = (ni > 0)

        transform_f1 = tf.layers.dense(
            masked_features,
            self.feature_dim * 2,
            name="Transform_f1",
            reuse=reuse_flag,
            use_bias=False)
        transform_f1 = tf.layers.batch_normalization(
            transform_f1,
            training=is_training,
            momentum=self.batch_momentum,
            virtual_batch_size=v_b)
        transform_f1 = glu(transform_f1, self.feature_dim)

        transform_f2 = tf.layers.dense(
            transform_f1,
            self.feature_dim * 2,
            name="Transform_f2",
            reuse=reuse_flag,
            use_bias=False)
        transform_f2 = tf.layers.batch_normalization(
            transform_f2,
            training=is_training,
            momentum=self.batch_momentum,
            virtual_batch_size=v_b)
        transform_f2 = (glu(transform_f2, self.feature_dim) +
                        transform_f1) * np.sqrt(0.5)

        transform_f3 = tf.layers.dense(
            transform_f2,
            self.feature_dim * 2,
            name="Transform_f3" + str(ni),
            use_bias=False)
        transform_f3 = tf.layers.batch_normalization(
            transform_f3,
            training=is_training,
            momentum=self.batch_momentum,
            virtual_batch_size=v_b)
        transform_f3 = (glu(transform_f3, self.feature_dim) +
                        transform_f2) * np.sqrt(0.5)

        transform_f4 = tf.layers.dense(
            transform_f3,
            self.feature_dim * 2,
            name="Transform_f4" + str(ni),
            use_bias=False)
        transform_f4 = tf.layers.batch_normalization(
            transform_f4,
            training=is_training,
            momentum=self.batch_momentum,
            virtual_batch_size=v_b)
        transform_f4 = (glu(transform_f4, self.feature_dim) +
                        transform_f3) * np.sqrt(0.5)

        if ni > 0:

          decision_out = tf.nn.relu(transform_f4[:, :self.output_dim])

          # Decision aggregation.
          output_aggregated += decision_out

          # Aggregated masks are used for visualization of the
          # feature importance attributes.
          scale_agg = tf.reduce_sum(
              decision_out, axis=1, keep_dims=True) / (
                  self.num_decision_steps - 1)
          aggregated_mask_values += mask_values * scale_agg

        features_for_coef = (transform_f4[:, self.output_dim:])

        if ni < self.num_decision_steps - 1:

          # Determines the feature masks via linear and nonlinear
          # transformations, taking into account of aggregated feature use.
          mask_values = tf.layers.dense(
              features_for_coef,
              self.num_features,
              name="Transform_coef" + str(ni),
              use_bias=False)
          mask_values = tf.layers.batch_normalization(
              mask_values,
              training=is_training,
              momentum=self.batch_momentum,
              virtual_batch_size=v_b)
          mask_values *= complemantary_aggregated_mask_values
          mask_values = tf.contrib.sparsemax.sparsemax(mask_values)

          # Relaxation factor controls the amount of reuse of features between
          # different decision blocks and updated with the values of
          # coefficients.
          complemantary_aggregated_mask_values *= (
              self.relaxation_factor - mask_values)

          # Entropy is used to penalize the amount of sparsity in feature
          # selection.
          total_entropy += tf.reduce_mean(
              tf.reduce_sum(
                  -mask_values * tf.log(mask_values + self.epsilon),
                  axis=1)) / (
                      self.num_decision_steps - 1)

          # Feature selection.
          masked_features = tf.multiply(mask_values, features)

          # Visualization of the feature selection mask at decision step ni
          tf.summary.image(
              "Mask for step" + str(ni),
              tf.expand_dims(tf.expand_dims(mask_values, 0), 3),
              max_outputs=1)

      # Visualization of the aggregated feature importances
      tf.summary.image(
          "Aggregated mask",
          tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3),
          max_outputs=1)

      return output_aggregated, total_entropy

  def classify(self, activations, reuse):
    """TabNet classify block."""

    with tf.variable_scope("Classify", reuse=reuse):
      logits = tf.layers.dense(activations, self.num_classes, use_bias=False)
      predictions = tf.nn.softmax(logits)
      return logits, predictions

  def regress(self, activations, reuse):
    """TabNet regress block."""

    with tf.variable_scope("Regress", reuse=reuse):
      predictions = tf.layers.dense(activations, 1)
      return predictions

# PyTorch implementation of TabNet

In [ ]:
! pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

# benchmark TabNet



In [ ]:
! python3 runbenchmark.py TabNet large

Running `TabNet` on `large` benchmarks in `local` mode.
Loading frameworks definitions from /content/gdrive/My Drive/AutoML_benchmark/automlbenchmark/resources/frameworks.yaml.
Loading benchmark constraint definitions from /content/gdrive/My Drive/AutoML_benchmark/automlbenchmark/resources/constraints.yaml.
Loading benchmark definitions from /content/gdrive/My Drive/AutoML_benchmark/automlbenchmark/resources/benchmarks/large.yaml.

------------------------------------------------
Starting job local.large.test.Airlines.0.TabNet.
[779] CPU Utilization: 4.0%
[779] Memory Usage: 8.4%
[779] Disk Usage: 22.1%
Assigning 2 cores (total=2) for new task Airlines.
Assigning 9853 MB (total=13021 MB) for new Airlines task.
Running task Airlines on framework TabNet with config:
TaskConfig(framework='TabNet', framework_params={}, framework_version='stable', type='classification', name='Airlines', fold=0, metrics=['auc', 'logloss', 'acc', 'balacc'], metric='auc', seed=579282976, max_runtime_seconds=60

For comparison

In [ ]:
! python3 runbenchmark.py DecisionTree large